## Khan Academy Math Student Dashboard: Data Cleaning
We will be creating a student dashboard that displays each student's progress through Khan Academy Math. The goal is to showcase:
- How the student is performing overall and this month?
- What is their assignment completion rate?
- Which topics are students strongest and weakest in?
- The students' percentile in the class?
- Is the student growing?

We want to automate the dashboarding process for each class in Lyceum Village, grades K-8. We will be testing on grade K-1 students' Khan Academy Math first to create functions and a webpage to showcase their progress.

### Import the Data
The data imported is for grade K-1. The students' names are already adjusted to an alias name.

In [52]:
import numpy as np
import pandas as pd
import datetime
import re

In [53]:
df = pd.read_csv("../Resources/K_1-Math-Anonymous.csv")

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Assignment Name              266 non-null    object 
 1   Student Name                 266 non-null    object 
 2   Score At Due Date            26 non-null     float64
 3   Score Best Ever              30 non-null     float64
 4   Points Possible              30 non-null     float64
 5   Number Of Attempts           266 non-null    int64  
 6   Most Recent Completion Date  30 non-null     object 
 7   Start Date                   266 non-null    object 
 8   Due Date                     266 non-null    object 
 9   Assignment URL               266 non-null    object 
 10  Assignment Type              266 non-null    object 
 11  Due Date (no time)           266 non-null    object 
dtypes: float64(3), int64(1), object(8)
memory usage: 25.1+ KB


Our csv dataset will have 11 variables.

Types of data:
- 3 float: 
    - Score At Due Date
    - Score Best Ever
    - Points Possible
- 1 integer:
    - Number Of Attempts
- 7 string:
    - Assignment Name
    - Student Name
    - Most Recent Completion Date
    - Start Date
    - Due Date
    - Assignment URL
    - Assignment Type

The data type so far matches what they should be.

There are 266 rows total. There are missing data in the following columns:
- Score At Due Date: 240 missing
- Score Best Ever: 224 missing
- Points Possible: 224 missing
- Most Recent Completion Date: 224 missing

They are missing data because some students did not complete their assignments. We will be removing these data in actual student analysis.

### Student Anonymity
We will need to create a student ID column to give each student anonymity on their scores when it is being displayed to other parents.
Their ID's will be formatted as so: grade_studentinitial_number <br>

Example: G2_KL_1

#### Create Student ID
We will be automating this system by having it done automatically only once. The application will request the grade level from the user and store the student ID information into a PostgreSQL database.

In [55]:
df['Student Name'].unique()

array(['Ned Ethans', 'Anna Kite', 'April Luna', 'Andy Hong',
       'Abby Nguyen', 'Everett Chase', 'Kyle Anderson', 'Kevin Martelle',
       'Misa Bing', 'Olsen Le', 'Terry Long'], dtype=object)

In [56]:
# User input grade level
stu_grade = input("Enter the student grade level: ")

In [57]:
# Loop through the list of unique students and create their ID
stu_id_list = []
for name in df['Student Name']:
    stu_name = name.split(" ")
    stu_id = "G" + stu_grade + "_" + stu_name[0][0] + stu_name[1][0]
    stu_id_list.append(stu_id)
stu_id_list[:5]

['GK1_NE', 'GK1_AK', 'GK1_AL', 'GK1_AH', 'GK1_AN']

In [58]:
# Add in the numeric value to account for students with the same initials
stu_id_list2 = []
if len(df['Student Name']) == len(stu_id_list):
    for id in stu_id_list:
        # Add 1 at the end because there are no repeated initials
        stu_id_list2.append(id + "1")
else:
    # change the number depending on the student name
    # current list of students do not have repeated initials. Create a dummy list to test this function on
    stu_id_list2
stu_id_list2[:5]

['GK1_NE1', 'GK1_AK1', 'GK1_AL1', 'GK1_AH1', 'GK1_AN1']

In [59]:
# Add id column to the front of the dataframe
df.insert(loc=0, column='Student ID', value=stu_id_list2)
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time)
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-comparing-numbers/e/comparing-numbers-through-10,Exercise,"Oct 8th,"
1,GK1_AK1,Add within 10,Anna Kite,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,"
2,GK1_AL1,Add within 10,April Luna,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,"
3,GK1_AH1,Add within 10,Andy Hong,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,"
4,GK1_AN1,Add within 10,Abby Nguyen,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,"


## Missing Data
NaN in the Score At Due Date, Score Best Ever, and Points Possible columns means that students did not complete their assignment. Thus, we will need to create a column that states the completion of their assignment.

Create a column with these values:
- Missing: Has 0 as the Number Of Attempts
- Complete: Has 1 or higher Number of Attempts and the Most Recent Completion Date is before or on the Due Date
- Late: Has 1 or more Number Of Attempts and the Most Recent Completion Date is past the Due Date

### Convert Date Columns to Date type
Convert the two columns from string object to date:
- Most Recent Completion Date
- Due Date

In [60]:
# Find the rows with no missing data
df_val = df[df['Most Recent Completion Date'].isna()==False]
df_val.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time)
96,GK1_KM1,Counting: Unit test,Kevin Martelle,13.0,13.0,13.0,2,"Sep 20th, 4:55PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc-early-math-counting-topic#cc-early-math-counting-topic-unit-test,Unit Test,"Sep 24th,"
100,GK1_TL1,Counting: Unit test,Terry Long,13.0,13.0,13.0,1,"Sep 20th, 4:42PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc-early-math-counting-topic#cc-early-math-counting-topic-unit-test,Unit Test,"Sep 24th,"
107,GK1_KM1,Add within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:01PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/addition_1,Exercise,"Sep 24th,"
111,GK1_TL1,Add within 5,Terry Long,7.0,7.0,7.0,1,"Sep 20th, 4:50PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/addition_1,Exercise,"Sep 24th,"
118,GK1_KM1,Subtract within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:04PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/subtraction_1,Exercise,"Sep 24th,"


In [61]:
# Function to convert the date from string type to date type
def convertDate(date_str, year):
    split_date = re.split(", | ", date_str) # Splits the date into Month, Day, Time
    day_num = int(re.findall('\d+', split_date[1])[0]) # Removes letters from the day. Example: 8th becomes 8
    month_num = datetime.datetime.strptime(split_date[0], '%b').month # Converts the month into the month's number
    
    # we are missing the year; we will have to have the user input it
    # Combined date
    date_str = str(month_num) + "/" + str(day_num) + "/" + year + " " + split_date[2]
    comb_date = datetime.datetime.strptime(date_str, '%m/%d/%Y %H:%M%p')

    return comb_date
    

In [62]:
progress_list = []
index_attempts = df.columns.get_loc('Number Of Attempts')
index_duedate = df.columns.get_loc('Due Date')
index_compdate = df.columns.get_loc('Most Recent Completion Date')
report_year = input('Year of report: ')
for i, r in df.iterrows():
    if r[index_attempts] == 0:
        progress_list.append('Missing')
    else:
        duedate_formatted = convertDate(r[index_duedate], report_year)
        compdate_formatted = convertDate(r[index_compdate], report_year)
        if duedate_formatted > compdate_formatted:
            progress_list.append('Complete')
        else:
            progress_list.append('Late')

In [63]:
# Add Progress column into the df
df['Progress'] = progress_list
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-comparing-numbers/e/comparing-numbers-through-10,Exercise,"Oct 8th,",Missing
1,GK1_AK1,Add within 10,Anna Kite,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
2,GK1_AL1,Add within 10,April Luna,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
3,GK1_AH1,Add within 10,Andy Hong,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
4,GK1_AN1,Add within 10,Abby Nguyen,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing


The Progress column which indicates if the assignment is missing, late, or complete. Now we can convert all missing data from the following columns into 0 if they are marked "Missing":
- Score At Due Date
- Score Best Ever

In [64]:
df.loc[df['Progress'] == 'Missing', ['Score At Due Date', 'Score Best Ever']] = 0
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-comparing-numbers/e/comparing-numbers-through-10,Exercise,"Oct 8th,",Missing
1,GK1_AK1,Add within 10,Anna Kite,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
2,GK1_AL1,Add within 10,April Luna,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
3,GK1_AH1,Add within 10,Andy Hong,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
4,GK1_AN1,Add within 10,Abby Nguyen,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing


Score At Due Date is zero for late assignments. Need to change all NaN to 0 if it is late.

In [65]:
df.loc[df['Progress'] == 'Late', ['Score At Due Date']] = 0

The following columns contains missing data for missed assignments, but cannot be set to 0:
- Points Possible: this is the total points received for the assignment. It should be set depending on the points available per assignment.
- Most Recent Completion Date: this will be left as missing as it is unrelated to scores

We need to find the Points Possible value for each assignment, and fill in the missing information.

In [66]:
df_no_missing = df.loc[df['Points Possible'].isna() == False]
df_no_missing.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
96,GK1_KM1,Counting: Unit test,Kevin Martelle,13.0,13.0,13.0,2,"Sep 20th, 4:55PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc-early-math-counting-topic#cc-early-math-counting-topic-unit-test,Unit Test,"Sep 24th,",Complete
100,GK1_TL1,Counting: Unit test,Terry Long,13.0,13.0,13.0,1,"Sep 20th, 4:42PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc-early-math-counting-topic#cc-early-math-counting-topic-unit-test,Unit Test,"Sep 24th,",Complete
107,GK1_KM1,Add within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:01PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/addition_1,Exercise,"Sep 24th,",Complete
111,GK1_TL1,Add within 5,Terry Long,7.0,7.0,7.0,1,"Sep 20th, 4:50PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/addition_1,Exercise,"Sep 24th,",Complete
118,GK1_KM1,Subtract within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:04PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/subtraction_1,Exercise,"Sep 24th,",Complete


In [67]:
# Check if the assignments in the dataframe with no missing values is the same for those that are missing.
df_nm_uniq = df.loc[df['Points Possible'].isna() == False, ['Assignment Name']]['Assignment Name'].unique()
df_nm_uniq

array(['Counting: Unit test', 'Add within 5', 'Subtract within 5',
       'Count objects 2', 'Compare numbers of objects 1',
       'Comparing numbers to 10', 'Counting: Quiz 2', 'Numbers to 100',
       'Numbers to 120', 'Count tens', 'Counting: Quiz 1',
       'Count with small numbers', 'Count in order',
       'Find 1 more or 1 less than a number', 'Missing numbers'],
      dtype=object)

In [68]:
# List of unique Assignment Names from values with missing data
df_m_uniq = df.loc[df['Points Possible'].isna() == True, ['Assignment Name']]['Assignment Name'].unique()
df_m_uniq

array(['Comparing numbers to 10', 'Add within 10', 'Subtract within 10',
       'Addition and subtraction intro: Quiz 2',
       'Relate addition and subtraction', 'Making 5',
       'Making small numbers in different ways',
       'Addition and subtraction intro: Quiz 1',
       'Make 10 (grids and number bonds)', 'Counting: Unit test',
       'Add within 5', 'Subtract within 5', 'Count objects 2',
       'Compare numbers of objects 1', 'Compare numbers of objects 2',
       'Counting: Quiz 2', 'Numbers to 100', 'Numbers to 120',
       'Count tens', 'Counting: Quiz 1', 'Count with small numbers',
       'Count in order', 'Find 1 more or 1 less than a number',
       'Missing numbers'], dtype=object)

In [69]:
print(f"Total Assignments that students did not complete: {len(df_m_uniq)}")
print(f"Total Assignments that students did complete: {len(df_nm_uniq)}")

Total Assignments that students did not complete: 24
Total Assignments that students did complete: 15


In [70]:
# Print out the assignment names that are missing and not in the completed assignment list
all_missing_assignments = []
for assignment in df_m_uniq:
    if assignment not in df_nm_uniq:
        all_missing_assignments.append(assignment)
all_missing_assignments

['Add within 10',
 'Subtract within 10',
 'Addition and subtraction intro: Quiz 2',
 'Relate addition and subtraction',
 'Making 5',
 'Making small numbers in different ways',
 'Addition and subtraction intro: Quiz 1',
 'Make 10 (grids and number bonds)',
 'Compare numbers of objects 2']

The assignments listed above are missing for all students in the class. Therefore, we would have to manually input the Points Possible. Thus, we will just fill in these assignments with 10 possible points.

In [71]:
# Fill in the Points Possible value as 10 for assignments that all students are missing
df.loc[df['Assignment Name'].isin(all_missing_assignments) == True, 'Points Possible'] = 10
df.head(20)

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-comparing-numbers/e/comparing-numbers-through-10,Exercise,"Oct 8th,",Missing
1,GK1_AK1,Add within 10,Anna Kite,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
2,GK1_AL1,Add within 10,April Luna,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
3,GK1_AH1,Add within 10,Andy Hong,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
4,GK1_AN1,Add within 10,Abby Nguyen,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
5,GK1_EC1,Add within 10,Everett Chase,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
6,GK1_KA1,Add within 10,Kyle Anderson,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
7,GK1_KM1,Add within 10,Kevin Martelle,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
8,GK1_MB1,Add within 10,Misa Bing,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing
9,GK1_NE1,Add within 10,Ned Ethans,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing


In [72]:
# Find the Points Possible for missing assignments where some students have completed it.
df_names_points = df_no_missing.loc[df_no_missing['Assignment Name'].isin(all_missing_assignments)==False, ['Assignment Name', 'Points Possible']].drop_duplicates()
df_names_points

,Assignment Name,Points Possible
96,Counting: Unit test,13.0
107,Add within 5,7.0
118,Subtract within 5,7.0
131,Count objects 2,7.0
142,Compare numbers of objects 1,7.0
153,Comparing numbers to 10,7.0
173,Counting: Quiz 2,6.0
178,Numbers to 100,7.0
192,Numbers to 120,7.0
206,Count tens,7.0


In [73]:
for index, row in df.iterrows():
    if row['Assignment Name'] in df_nm_uniq:
        print(df['Points Possible'][index])
        print(df_names_points.loc[df_names_points['Assignment Name'] == df['Assignment Name'][index], 'Points Possible'])
    break

nan
153    7.0
Name: Points Possible, dtype: float64


In [74]:
# Fill in missing values with the correct corresponding Points Possible
for index, row in df.iterrows():
    if row['Assignment Name'] in df_nm_uniq:
        df['Points Possible'][index] = df_names_points.loc[df_names_points['Assignment Name'] == df['Assignment Name'][index], 'Points Possible']
df.loc[df['Assignment Name'].isin(df_nm_uniq) == True, ['Assignment Name', 'Points Possible']]

C:\Users\krist\AppData\Local\Temp\ipykernel_21500\1929856247.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Points Possible'][index] = df_names_points.loc[df_names_points['Assignment Name'] == df['Assignment Name'][index], 'Points Possible']


,Assignment Name,Points Possible
0,Comparing numbers to 10,7.0
89,Comparing numbers to 10,7.0
90,Counting: Unit test,13.0
91,Counting: Unit test,13.0
92,Counting: Unit test,13.0
...,...,...
261,Missing numbers,7.0
262,Missing numbers,7.0
263,Missing numbers,7.0
264,Missing numbers,7.0


In [75]:
# Check rows to see if the missing numbers were filled properly for Possible Points
df.loc[df['Assignment Name'].isin(df_nm_uniq)==True, ['Assignment Name', 'Points Possible']].drop_duplicates()

,Assignment Name,Points Possible
0,Comparing numbers to 10,7.0
90,Counting: Unit test,13.0
101,Add within 5,7.0
112,Subtract within 5,7.0
123,Count objects 2,7.0
134,Compare numbers of objects 1,7.0
167,Counting: Quiz 2,6.0
178,Numbers to 100,7.0
189,Numbers to 120,7.0
200,Count tens,7.0


In [76]:
# Check if there are any missing values in Possible Points
df['Points Possible'].isna().sum()

0

## Add Assignment Grade Level Column
Although the class is filled with kindergarten and 1st graders, the assignments vary in difficulty. We will need to find the unique assignment levels from the URL.

In [77]:
# Allows us to print entire value without truncating
pd.set_option('display.max_colwidth', -1)
print(df["Assignment URL"].head(5))

0    https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-comparing-numbers/e/comparing-numbers-through-10
1    https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together               
2    https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together               
3    https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together               
4    https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together               
Name: Assignment URL, dtype: object


C:\Users\krist\AppData\Local\Temp\ipykernel_21500\1367671070.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


The unit topics are located after "cc-kindergarten-". We can extract to collect the unit topic by splitting the URL using "cc-".

In [78]:
unit_topic = []
for rows in df["Assignment URL"]:
    unit_topic.append(re.split("/cc-|#cc-|/", rows)[5])

In [79]:
uniq_unit_topic = set(unit_topic)
print(uniq_unit_topic)

{'early-math-add-sub-basics', '1st-add-subtract', 'kindergarten-counting', '1st-place-value', 'kindergarten-add-subtract', 'early-math-counting-topic'}


The different levels of difficulty are displayed as the first word in the string, and then the unit topic follows after. We will now split this string and add them to the dataframe as two new columns called "Assignment Grade Level" and "Unit Topic".

In [80]:
level_list = []
unit_list = []

for rows in unit_topic:
    if("early-math" in rows):
        pattern = r'^([\w]+-[\w]+)'
        row_split = re.match(pattern, rows).group()
        level_list.append(row_split)
        
        if ("add-sub" in rows[len(row_split)+1:]):
            unit_list.append("add-subtract")
        elif ("counting" in rows[len(row_split)+1:]):
            unit_list.append("counting")
    else:
        row_split = rows.split("-", 1)
        level_list.append(row_split[0])
        unit_list.append(row_split[1])

print(len(level_list), len(unit_list))
level_set = set(level_list)
unit_set = set(unit_list)
print(level_set)
print(unit_set)

266 266
{'kindergarten', 'early-math', '1st'}
{'counting', 'add-subtract', 'place-value'}


In [81]:
# Add Grade Level and Unit Topic to dataframe
df['Assignment Grade Level'] = level_list
df['Unit Topic'] = unit_list
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress,Assignment Grade Level,Unit Topic
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,0.0,0.0,7.0,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-comparing-numbers/e/comparing-numbers-through-10,Exercise,"Oct 8th,",Missing,kindergarten,counting
1,GK1_AK1,Add within 10,Anna Kite,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing,kindergarten,add-subtract
2,GK1_AL1,Add within 10,April Luna,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing,kindergarten,add-subtract
3,GK1_AH1,Add within 10,Andy Hong,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing,kindergarten,add-subtract
4,GK1_AN1,Add within 10,Abby Nguyen,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-together-apart/e/put-together,Exercise,"Oct 8th,",Missing,kindergarten,add-subtract


## Weighted Scores
Khan Academy organizes student progress based on completion. However, this doesn't tell much about students' math comprehension. Therefore, we will be creating a measure to more accurately gauge student comprehension. Therefore, each score will be weighted by the number of attempts for each assignment.

Formula:

$$
Weighted Score = Actual Score (1 + \frac{Class Avg Attempts - Student Total Attempts}{Class Avg Attempts})
$$ (my_other_label)

Where ClassAvgAttempts is the average number of attempts for the assignment.

In [82]:
# Find the ClassAvgAttempts per assignment
# Ignore the rows where students did not complete the assignment
df_mean = df.loc[df['Progress'] != 'Missing'].groupby(['Assignment Name']).mean()
df_mean

,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts
Assignment Name,,,,
Add within 5,7.000000,7.000000,7.0,1.000000
Compare numbers of objects 1,6.500000,6.500000,7.0,2.500000
Comparing numbers to 10,4.000000,4.000000,7.0,1.000000
Count in order,4.666667,7.000000,7.0,2.000000
Count objects 2,7.000000,7.000000,7.0,1.000000
Count tens,7.000000,7.000000,7.0,1.000000
Count with small numbers,4.666667,6.666667,7.0,1.000000
Counting: Quiz 1,0.000000,7.000000,7.0,2.000000
Counting: Quiz 2,0.000000,6.000000,6.0,1.000000


Let's create a function that standardizes scores based on the averages. We will call this function `weight_Score_attempts`. This function will return a new dataframe with the score weighted by the average of attempts.
<br>`weight_Score_attempts(df, mean_df)`
<br>Arguments:
- `df` (dataframe): dataframe that includes the student scores.
- `mean_df` (dataframe): dataframe that includes the mean values of all numerical columns


In [125]:
def weight_Score_attempts(df, mean_df):
    df_new = df.merge(mean_df[['Number Of Attempts']], on= "Assignment Name", how='outer')    
    df_new['Weighted Score'] = (1 + (df_new['Number Of Attempts_y'] - df_new['Number Of Attempts_x'])/df_new['Number Of Attempts_y'])*df_new["Score Best Ever"]
    # Have any missing data for Number Of Attempts_y and Weighted Score set to 0
    df_new.fillna({'Number Of Attempts_y':0, 'Weighted Score': 0}, inplace=True)
    df_new.columns= list(df.columns) + ['Average Number Of Attempts', 'Weighted Score']
    return df_new

In [126]:
df_weight = weight_Score_attempts(df, df_mean)

In [127]:
len(df_weight.loc[df_weight['Weighted Score'].isna() == True,])

0

In [129]:
df_weight.sample(10)

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress,Assignment Grade Level,Unit Topic,Average Number Of Attempts,Weighted Score
51,GK1_KA1,Relate addition and subtraction,Kyle Anderson,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-1st-grade-math/cc-1st-add-subtract/cc-1st-add-subtract-10/e/relate-addition-and-subtraction,Exercise,"Oct 8th,",Missing,1st,add-subtract,0.0,0.000000
120,GK1_NE1,Add within 5,Ned Ethans,0.0,0.0,7.0,0,NaN,"Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/addition_1,Exercise,"Sep 24th,",Missing,kindergarten,add-subtract,1.0,0.000000
265,GK1_TL1,Missing numbers,Terry Long,6.0,6.0,7.0,1,"Aug 30th, 4:47PM","Aug 25th, 9:35AM","Sep 3rd, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-numbers-100/e/count-from-any-number,Exercise,"Sep 3rd,",Complete,kindergarten,counting,2.0,9.000000
93,GK1_AN1,Make 10 (grids and number bonds),Abby Nguyen,0.0,0.0,10.0,0,NaN,"Sep 29th, 4:48PM","Oct 1st, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-making-5-9/e/making-ten,Exercise,"Oct 1st,",Missing,kindergarten,add-subtract,0.0,0.000000
194,GK1_KA1,Numbers to 120,Kyle Anderson,0.0,0.0,7.0,0,NaN,"Sep 1st, 1:04PM","Sep 10th, 11:59PM",https://www.khanacademy.org/math/cc-1st-grade-math/cc-1st-place-value/cc-1st-numbers-120/e/numbers-to-120,Exercise,"Sep 10th,",Missing,1st,place-value,1.5,0.000000
239,GK1_KM1,Count in order,Kevin Martelle,7.0,7.0,7.0,2,"Aug 30th, 4:36PM","Aug 25th, 9:35AM","Sep 3rd, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-counting/e/counting-objects,Exercise,"Sep 3rd,",Complete,kindergarten,counting,2.0,7.000000
116,GK1_EC1,Add within 5,Everett Chase,0.0,0.0,7.0,0,NaN,"Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-add-subtract/kindergarten-add-sub-intro/e/addition_1,Exercise,"Sep 24th,",Missing,kindergarten,add-subtract,1.0,0.000000
141,GK1_MB1,Count objects 2,Misa Bing,0.0,0.0,7.0,0,NaN,"Sep 3rd, 1:47PM","Sep 17th, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-count-object-topic/e/how-many-objects-2,Exercise,"Sep 17th,",Missing,kindergarten,counting,1.0,0.000000
258,GK1_AN1,Missing numbers,Abby Nguyen,0.0,0.0,7.0,0,NaN,"Aug 25th, 9:35AM","Sep 3rd, 11:59PM",https://www.khanacademy.org/math/cc-kindergarten-math/cc-kindergarten-counting/kindergarten-numbers-100/e/count-from-any-number,Exercise,"Sep 3rd,",Missing,kindergarten,counting,2.0,0.000000
111,GK1_TL1,Counting: Unit test,Terry Long,13.0,13.0,13.0,1,"Sep 20th, 4:42PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc-early-math-counting-topic#cc-early-math-counting-topic-unit-test,Unit Test,"Sep 24th,",Complete,early-math,counting,1.5,17.333333


## Remove Unnecessary Columns
Now that our dataset it clean, let's remove any unnecessary columns before importing it into a SQL database. The columns that will need to be removed are:
- Score At Due Date: We can remove this as we will be focusing on their Best Score Ever
- Assignment URL: We have extracted the necessary information from the URL already so we no longer need this section
- Due Date (no time): We already have a Due Date column so this section is now unnecessary